# All necessary imports

In [ ]:

# General python Imports
import os
import sys
import numpy as np
from numpy import array, nan

from functools import partial
from importlib import reload
from time import sleep, time
from matplotlib import pyplot as plt


# Qcodes imports (general measurement code structure)
import qcodes as qc
from qcodes import Instrument
from qcodes.instrument.parameter import Parameter, ManualParameter, StandardParameter

# SilQ imports (specific measurements and setup)
from silq.parameters import measurement_parameters, general_parameters
from silq.instrument_interfaces import get_instrument_interface


# General functions (maybe this should be somewhere else?)

In [2]:
def plot_traces(traces, traces_AWG=None, threshold_voltage=None, plot1D=False):
    '''
    plots the measured traces of the digitizer (traces), 
    the AWG voltage levels (traces_AWG, if not none) 
    and the threshold voltage (threshold_voltage, if not none) which determines up or down spin
    '''
    plt.figure()
    plt.pcolormesh(range(traces.shape[1]),
                   range(traces.shape[0] + 1), traces)
    if traces_AWG is not None:
        trace_AWG = traces_AWG[:1]
        trace_AWG /= (np.max(trace_AWG) - np.min(trace_AWG))
        trace_AWG -= np.min(trace_AWG)
        plt.pcolormesh(range(traces.shape[1]),
                       np.array([0, 1]) + traces.shape[0], trace_AWG)
    plt.xlim([0, traces.shape[1]])
    plt.ylim([0, traces.shape[0] + 1])
    plt.gca().invert_yaxis()
    plt.colorbar()

    if plot1D:
        fig, axes = plt.subplots(len(traces), sharex=True)
        for k, trace in enumerate(traces):
            axes[k].plot(trace)
            #         axes[k].plot(trace > 0.5)
            if traces_AWG is not None:
                trace_AWG = traces_AWG[k]
                trace_AWG /= (np.max(trace_AWG) - np.min(trace_AWG))
                trace_AWG -= np.min(trace_AWG)
                axes[k].plot(trace_AWG)
            if threshold_voltage is not None:
                axes[k].plot([threshold_voltage] * len(trace), 'r')
            axes[k].locator_params(nbins=2)


def try_close_instruments(
        instruments=['pulseblaster', 'arbstudio', 'SIM900', 'ATS',
                     'ATS_controller', 'pulsemaster'],
        reload=False):
    '''
    closes the instrument instance in python 
    '''
    if isinstance(instruments, str):
        instruments = [instruments]
    for instrument_name in instruments:
        try:
            eval('{}.close()'.format(instrument_name))
        except:
            pass
        try:
            eval('reload({}_driver)'.format(instrument_name))
        except:
            pass


def print_voltages(SIM900):
    '''
    returns the voltage levels for all SIM-rack channels (at the top of the fridge)
    '''
    for channel, name in SIM900.channels().items():
        print('{}({})'.format(name, SIM900.parameters[name]()))


def ramp_to_voltages(target_voltages, SIM900=None, channels=None):
    '''
    savely ramps the voltage of the SIM-rack channels to avoid fast voltage changes
    rate: 10% of the voltage difference at once
    '''
    if channels is None:
        channels = [SIM900.parameters[ch_name] for ch_name in
                    SIM900.channels().values()]

    if isinstance(target_voltages, int):
        target_voltages = {channel.name: target_voltages for channel in
                           channels}

    initial_voltages = {channel.name: channel() for channel in channels}
    for ratio in np.linspace(0, 1, 11):
        for channel in channels:
            voltage = (1 - ratio) * initial_voltages[channel.name] + \
                      ratio * target_voltages[channel.name]
            channel(voltage)

# Instruments

In [ ]:
# Load specific instruments of the current setup
from qcodes.instrument_drivers.AlazarTech.ATS9440 import ATS9440 #Digitizer
from qcodes.instrument_drivers.AlazarTech.ATS_acquisition_controllers import \
    Triggered_AcquisitionController, Continuous_AcquisitionController #Controller for Digitizer
from qcodes.instrument_drivers.keysight.E8267D import Keysight_E8267D #MW source
from qcodes.instrument_drivers.lecroy.ArbStudio1104 import ArbStudio1104 #AWG
from qcodes.instrument_drivers.spincore.PulseBlasterESRPRO import PulseBlasterESRPRO #Pulse generator (mainly for triggering)
from qcodes.instrument_drivers.stanford_research.SIM900 import SIM900 #Sim-rack

# instrument placeholder representing the sample chip
from silq.meta_instruments.chip import Chip 
# incoorporates a scaling factor into the parameters, e.g. to accomodate for voltage dividers
from silq.parameters.general_parameters import ScaledParameter

#Instrument interfaces to the main code
interfaces = {}


####### All the instrument initializations

### SIM900
SIM900 = SIM900('SIM900', 'GPIB0::4::INSTR',server_name='')
# Each DC voltage source has format (name, slot number, divider, max raw voltage)
DC_sources = [('TG',1,8,18), ('LB',2,4,8), ('RB',3,4,8), ('TGAC',4,5,4),
         ('SRC',5,1,1), ('DS',7,4,3.2), ('DF',6,4,3.2)]
SIM900_scaled_parameters = []
for ch_name, ch, ratio,max_voltage in DC_sources:
    #set divider scaling parameter and maximum voltage for safety
    SIM900.define_slot(channel=ch, name=ch_name+'_raw', max_voltage=max_voltage)
    SIM900.update()
    param_raw = SIM900.parameters[ch_name+'_raw'] #voltage at the SIM-rack (on top of the fridge)
    param = ScaledParameter(param_raw, name=ch_name, label=ch_name, ratio=ratio) # voltage after the divider (at the sample)
    SIM900_scaled_parameters.append(param)

    exec('{ch_name}_raw = param_raw'.format(ch_name=ch_name))
    exec('{ch_name} = param'.format(ch_name=ch_name))


### ArbStudio
dll_path = os.path.join(os.getcwd(),'C:\lecroy_driver\\Library\\ArbStudioSDK.dll')
arbstudio = ArbStudio1104('arbstudio',
                          dll_path=dll_path,
                          server_name='' if USE_MP else None)
for ch in ['ch1', 'ch2', 'ch3', 'ch4']:
    arbstudio.parameters[ch + '_sampling_rate_prescaler'](250)
interfaces['arbstudio'] = get_instrument_interface(arbstudio)


### PulseBlaster
pulseblaster = PulseBlasterESRPRO('pulseblaster',
                            api_path='spinapi.py',
                            server_name='' if USE_MP else None)
pulseblaster.core_clock(500)
interfaces['pulseblaster'] = get_instrument_interface(pulseblaster)


### Chip
chip = Chip(name='chip', server_name='' if USE_MP else None)
interfaces['chip'] = get_instrument_interface(chip)


### ATS
ATS = ATS9440('ATS', server_name='Alazar_server' if USE_MP else None)
triggered_controller = Triggered_AcquisitionController(
    name='triggered_controller',
    alazar_name='ATS',
    server_name='Alazar_server' if USE_MP else None)
continuous_controller = Continuous_AcquisitionController(
    name='continuous_controller',
    alazar_name='ATS',
    server_name='Alazar_server' if USE_MP else None)
interfaces['ATS'] = get_instrument_interface(ATS)
interfaces['ATS'].add_acquisition_controller('triggered_controller')
interfaces['ATS'].add_acquisition_controller('continuous_controller')


### MW source
keysight = Keysight_E8267D('keysight','TCPIP0::192.168.0.5::inst0::INSTR',
                           server_name='' if USE_MP else None)
interfaces['keysight'] = get_instrument_interface(keysight)
interfaces['keysight'].modulation_channel('ext2')
interfaces['keysight'].envelope_padding(0.2)
keysight.power(10)